<a href="https://www.kaggle.com/code/aisuko/masked-language-modelling-nlp?scriptVersionId=164636357" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

Maksed language modeling predicts a maksed token in a sequence, and the model can attend tokens bidirectionally. This means the model has full access to the tokens on the left and right. Masked language modeling is great for tasks that require a good contextual understanding of an entire sequence. BERT is an example of a masked language model. Here we are going to fine-tune DistilRoBERTa( a full-mask model) on a (Text2Text Generation) datasets. 

In [1]:
%%capture
!pip install transformers==4.35.2
!pip install datasets==2.15.0
!pip install evaluate==0.4.1

In [2]:
import os
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

login(token=user_secrets.get_secret("HUGGINGFACE_TOKEN"))

os.environ["WANDB_API_KEY"]=user_secrets.get_secret("WANDB_API_KEY")
os.environ["WANDB_PROJECT"] = "Fine-tune-models"
os.environ["WANDB_NOTES"] = "Fine tune model distilbert base uncased"
os.environ["WANDB_NAME"] = "ft-distilroberta-base-with-askscience"

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load ELI5 dataset

Note: we use the `r/askscience` which is a subset of the ELI5 dataset.

And here we load a smaller subset from dataset. This will give us a chance to experiment and make sure everything works before spending more time training on the full dataset.

In [3]:
from datasets import load_dataset

eli5=load_dataset("eli5_category", split="train[:5000]")
eli5

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

Dataset({
    features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
    num_rows: 5000
})

Split the dataset's `train_asks` split into a train and test set with the `train_test_split` method:

In [4]:
eli5 =eli5.train_test_split(test_size=0.2)
eli5

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
        num_rows: 1000
    })
})

In [5]:
eli5["train"][0]

{'q_id': '5q1dsi',
 'title': 'Why is the night sky more clear after it rains?',
 'selftext': '',
 'category': 'Other',
 'subreddit': 'explainlikeimfive',
 'answers': {'a_id': ['dcvlbm5'],
  'text': ['The air is full of dust, pollen, dander, debris and dirt kicked up from the ground, haze from automobile exhaust, smoke, and factories, and the rain wipes it all clean as it falls the same way it washes your dishes. Dirt and debris cling to the drops and it delivers them to the ground clearing the air as it falls.'],
  'score': [4],
  'text_urls': [[]]},
 'title_urls': ['url'],
 'selftext_urls': ['url']}

The language modeling tasks is an unsupervised task, the next word is the label. And you will notice from the example above, the `text` filed is actually nested inside `answer`. This means we will need to extract the text subfiled from its nested structure with the `flatten` method.

In [6]:
eli5=eli5.flatten()
eli5["train"]

Dataset({
    features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'answers.text_urls', 'title_urls', 'selftext_urls'],
    num_rows: 4000
})

# Preprocess

We can see that each subfiled is now a separate column as indicated by the `answers` prefix, and the text filed is a list now. Instead of tokenizing each sentence separately, convert the list to a string so we can jointly tokenize them. 

In [7]:
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained("distilroberta-base")
print(tokenizer)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

RobertaTokenizerFast(name_or_path='distilroberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}


In [8]:
# Function to join the list of strings for each example and tokenize the result
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

tokenized_eli5=eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

print(tokenized_eli5)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (648 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1125 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (995 > 512). Running this sequence through the model will result in indexing errors


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2644 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})


Split the concatenated sequences into shorter chunks defined by `block_size`, which should be both shorter than the maximum input length and short enough for the GPU RAM.

In [9]:
block_size=128

def group_texts(examples):
    # Concatentate all texts
    concatenated_examples={k: sum(examples[k], []) for k in examples.keys()}
    total_length=len(concatenated_examples[list(examples.keys())[0]])
    
    if total_length>block_size:
        total_length=(total_length//block_size)* block_size
    
    # Split by chunks of block_size
    result={
        k: [t[i: i+block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    return result

In [10]:
# Apply the group_texts function over the entire dataset

lm_dataset=tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Here we will use the `dynamically pad` the sentences to the longest length in a batch during collation and the specify `mlm_probability` to randomly mask tokens rach time we iterate over the data.

In [11]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token=tokenizer.eos_token
data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# Training



In [12]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer

model=AutoModelForMaskedLM.from_pretrained("distilroberta-base")
print(model.config)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



In [13]:
training_args=TrainingArguments(
    output_dir=os.getenv("WANDB_NAME"),
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_checkpointing=True,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=True, # mixed 
    report_to="wandb",
    run_name=os.getenv("WANDB_NAME"),
    push_to_hub=False,
)


trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

wandb: Currently logged in as: urakiny (causal_language_trainer). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240228_050214-wealp8dh
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ft-distilroberta-base-with-askscience
wandb: ⭐️ View project at https://wandb.ai/causal_language_trainer/Fine-tune-models
wandb: 🚀 View run at https://wandb.ai/causal_language_trainer/Fine-tune-models/runs/wealp8dh
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all inpu

Epoch,Training Loss,Validation Loss
1,No log,2.049015
2,2.250400,2.050697


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


TrainOutput(global_step=672, training_loss=2.2341062000819614, metrics={'train_runtime': 719.0162, 'train_samples_per_second': 29.874, 'train_steps_per_second': 0.935, 'total_flos': 712179101399040.0, 'train_loss': 2.2341062000819614, 'epoch': 2.0})

In [14]:
import math

eval_results=trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Perplexity: 7.96


In [15]:
tokenizer.push_to_hub(os.getenv("WANDB_NAME"))
trainer.push_to_hub(os.getenv("WANDB_NAME"))

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.22k [00:00<?, ?B/s]

'https://huggingface.co/aisuko/ft-distilroberta-base-with-askscience/tree/main/'

# Inference

In [16]:
from transformers import pipeline

text="The Miky Way is a <mask> galaxy."
mask_filter =pipeline("fill-mask", model=os.getenv("WANDB_NAME"))
mask_filter(text, top_k=3)

[{'score': 0.14693552255630493,
  'token': 21300,
  'token_str': ' spiral',
  'sequence': 'The Miky Way is a spiral galaxy.'},
 {'score': 0.05938957259058952,
  'token': 650,
  'token_str': ' small',
  'sequence': 'The Miky Way is a small galaxy.'},
 {'score': 0.054233890026807785,
  'token': 2232,
  'token_str': ' massive',
  'sequence': 'The Miky Way is a massive galaxy.'}]

Tokenize the text and return the `input_ids` as PyTorch tensors. You will also need to specify the position of the `<mask>` token:

In [17]:
import torch
from transformers import AutoTokenizer

tokenizer=AutoTokenizer.from_pretrained(os.getenv("WANDB_NAME"))
inputs=tokenizer(text, return_tensors="pt")
mask_token_index=torch.where(inputs["input_ids"]==tokenizer.mask_token_id)[1]

Pass the inputs to the model and return the logits of the masked token

In [18]:
from transformers import AutoModelForMaskedLM

model=AutoModelForMaskedLM.from_pretrained(os.getenv("WANDB_NAME"))
logits=model(**inputs).logits
mask_token_logits=logits[0, mask_token_index,:]

Then return the three masked tokens with the highest probability and print them out.

In [19]:
top_3_tokens=torch.topk(mask_token_logits,3, dim=1).indices[0].tolist()

for token in top_3_tokens:
    print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

The Miky Way is a  spiral galaxy.
The Miky Way is a  small galaxy.
The Miky Way is a  massive galaxy.
